In [1]:
# -*- coding: utf-8 -*-

# Importação das bibliotecas necessárias
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import mlflow
import mlflow.sklearn

In [2]:
def carregar_dados_iris():
    """
    Carrega o dataset Iris do scikit-learn.
    """
    iris = load_iris()
    X = pd.DataFrame(iris.data, columns=iris.feature_names)
    y = pd.Series(iris.target, name='species')
    # Para facilitar a visualização, vamos usar os nomes das espécies
    nomes_especies = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
    y = y.map(nomes_especies)
    return X, y, iris.target_names

def gerar_e_salvar_matriz_confusao(y_true, y_pred, target_names, nome_arquivo="matriz_confusao.png"):
    """
    Gera uma Matriz de Confusão usando Seaborn e a salva como um arquivo de imagem.
    """
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=target_names, yticklabels=target_names)
    plt.xlabel('Previsto')
    plt.ylabel('Verdadeiro')
    plt.title('Matriz de Confusão')
    plt.savefig(nome_arquivo)
    plt.close()
    print(f"Matriz de confusão salva como '{nome_arquivo}'")
    return nome_arquivo



In [3]:
# --- Execução Principal e Integração com MLflow ---

if __name__ == "__main__":
    # Carregar os dados
    X, y, target_names = carregar_dados_iris()

    # Dividir os dados em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

    # Definindo o nome do experimento no MLflow
    mlflow.set_experiment("Classificacao_Iris_RedeNeural")

    # --- Início dos Experimentos ---
    # Vamos variar a arquitetura da rede neural
    # Cada tupla representa as camadas ocultas e o número de neurônios em cada uma
    arquiteturas = [(50,), (100,), (50, 50), (100, 50, 25)]
    max_iter_param = 500 # Aumentar o número de iterações para garantir a convergência

    print("Iniciando execuções de experimentos com MLflow...")

    for hidden_layers in arquiteturas:
        with mlflow.start_run():
            run_name = f"run_layers_{hidden_layers}"
            mlflow.set_tag("mlflow.runName", run_name)
            print(f"Executando: {run_name}")

            # Registrando os parâmetros
            mlflow.log_param("hidden_layer_sizes", hidden_layers)
            mlflow.log_param("max_iter", max_iter_param)
            mlflow.log_param("random_state", 42)

            # Criando e treinando o modelo de Rede Neural
            mlp = MLPClassifier(hidden_layer_sizes=hidden_layers, max_iter=max_iter_param, random_state=42)
            mlp.fit(X_train, y_train)

            # Realizando as previsões
            y_pred = mlp.predict(X_test)

            # Calculando e registrando a métrica de acurácia
            acuracia = accuracy_score(y_test, y_pred)
            print(f"Acurácia para arquitetura {hidden_layers}: {acuracia:.4f}")
            mlflow.log_metric("acuracia", acuracia)

            # Gerando e salvando a matriz de confusão como artefato
            caminho_matriz = gerar_e_salvar_matriz_confusao(y_test, y_pred, target_names)
            mlflow.log_artifact(caminho_matriz, "visualizacoes")
            print(f"Artefato '{caminho_matriz}' registrado.")

            # Opcional: Salvar o relatório de classificação como um arquivo de texto
            report = classification_report(y_test, y_pred, target_names=target_names)
            mlflow.log_text(report, "classification_report.txt")
            print("Relatório de classificação registrado.")

            # Registrando o modelo
            mlflow.sklearn.log_model(mlp, "mlp_model")

            print("-" * 40)

    print("Todas as execuções foram concluídas.")
    print("Para visualizar os resultados, execute 'mlflow ui' no seu terminal.")

2025/06/28 16:05:45 INFO mlflow.tracking.fluent: Experiment with name 'Classificacao_Iris_RedeNeural' does not exist. Creating a new experiment.


Iniciando execuções de experimentos com MLflow...
Executando: run_layers_(50,)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Acurácia para arquitetura (50,): 1.0000
Matriz de confusão salva como 'matriz_confusao.png'
Artefato 'matriz_confusao.png' registrado.
Relatório de classificação registrado.


2025/06/28 16:05:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

----------------------------------------
Executando: run_layers_(100, 50, 25)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


Acurácia para arquitetura (100, 50, 25): 0.9111
Matriz de confusão salva como 'matriz_confusao.png'
Artefato 'matriz_confusao.png' registrado.
Relatório de classificação registrado.


2025/06/28 16:06:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


----------------------------------------
Todas as execuções foram concluídas.
Para visualizar os resultados, execute 'mlflow ui' no seu terminal.
